In [1]:
from pydent_helper import *

In [3]:
session = get_session("production")

Connected to Aquarium at http://52.27.43.242/ using pydent version 0.0.35
Logged in as Devin Strickland



In [5]:
st = session.SampleType.find_by_name("DNA Library")

In [6]:
samples = st.samples
print(len(samples))

194


In [10]:
sample_ids = [s.id for s in samples]
items = []
for sample_id in sample_ids:
    these_items = []
    try:
        these_items = session.Item.where({"sample_id": sample_id})
        print("Found {} Items for Sample {}".format(len(these_items), sample_id))
    except Exception as e: 
        print("### Sample {}:".format(sample_id,e))
           
    items.append(these_items)

Found 474 Items for Sample 22088
Found 42 Items for Sample 22235
Found 266 Items for Sample 22312
Found 7 Items for Sample 22328
Found 7 Items for Sample 22329
Found 400 Items for Sample 22610
Found 12 Items for Sample 22611
Found 11 Items for Sample 22612
Found 154 Items for Sample 22613
Found 337 Items for Sample 22614
Found 25 Items for Sample 22776
Found 25 Items for Sample 22777
Found 9 Items for Sample 22806
Found 90 Items for Sample 23161
Found 409 Items for Sample 23166
Found 219 Items for Sample 23211
Found 24 Items for Sample 23244
Found 141 Items for Sample 23245
Found 105 Items for Sample 23247
Found 213 Items for Sample 23294
Found 43 Items for Sample 23295
Found 46 Items for Sample 23296
Found 46 Items for Sample 23297
Found 67 Items for Sample 23298
Found 66 Items for Sample 23299
Found 90 Items for Sample 23300
Found 252 Items for Sample 23301
Found 4 Items for Sample 23553
Found 38 Items for Sample 23684
Found 40 Items for Sample 23685
Found 39 Items for Sample 23686
F

In [11]:
print(len(items))

194


In [12]:
flat_items = [item for sublist in items for item in sublist]
print(len(flat_items))

19771


In [13]:
pacific = timezone('US/Pacific')

window_end = pacific.localize(datetime(2019, 10, 13))
print(window_end)

2019-10-13 00:00:00-07:00


In [14]:
unstored_locations = [
    'ice bucket', 
    '30 C shaker incubator', 
    'Culture discard area', 
    '30 °C shaker incubator', 
    '30 C shaker', 
    'Bench'
]

In [15]:
unstored_items = [i for i in flat_items if i.location in unstored_locations]
print(len(unstored_items))

364


In [16]:
old_unstored_items = [i for i in unstored_items if dateutil.parser.parse(i.updated_at) < window_end]
print(len(old_unstored_items))

349


In [17]:
ot_ct = Counter()

for i in old_unstored_items:
    ot_ct[i.object_type_id] += 1
    
print(ot_ct)

Counter({747: 338, 748: 8, 746: 3})


In [18]:
ot_ids = list(ot_ct.keys())
print(ot_ids)
ots = session.ObjectType.find(ot_ids)
for ot in ots:
    print(str(ot.id) + " " + ot.name)

[748, 747, 746]
746 Yeast Library Glycerol Stock
747 Labeled Yeast Library Suspension
748 Yeast Library Liquid Culture


In [20]:
old_unstored_items = [i for i in old_unstored_items if i.object_type_id != 746]
print(len(old_unstored_items))

346


In [21]:
def json_update(model, **params):
    """This update method is fairly dangerous. Be careful!"""
    data = {"model": {"model": model.__class__.__name__}}
    data.update(model.dump(**params))
    return model.session.utils.aqhttp.post('json/save', json_data=data)

In [23]:
for item in old_unstored_items:
    item.location = 'deleted'
    item.quantity = 0
    json_update(item)